In [1]:
import os
from pathlib import Path
os.chdir(Path.cwd().parent)
# print("cwd is now:", Path.cwd())

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from data_loader import scores_df
from fit import fit_statistical_model

null performances after coercion: 0
after saturation filter 995
after filter num benchmarks 735
after merge with model versions 740
after merge with benchmark dates 740
Original number of rows: 740
Number of rows after aggregation: 658


In [6]:
anchor_benchmark = "Winogrande"
anchor_difficulty = 0
anchor_slope = 1
df_anchor, df_cm_anchor, df_db_anchor = fit_statistical_model(scores_df, anchor_benchmark, anchor_difficulty, anchor_slope)

# Convert date strings to datetime objects
df_cm_anchor['date_obj'] = pd.to_datetime(df_cm_anchor['date'])

`ftol` termination condition is satisfied.
Function evaluations 36, initial cost 2.3912e+01, final cost 1.6297e+00, first-order optimality 7.03e-05.


In [9]:
df_cm_anchor

,model_id,estimated_capability,model,Model,date,date_obj
65,m60,2.028895,o3-2025-04-16_high,o3,2025-04-16,2025-04-16
80,m74,2.024028,o3-2025-04-16_medium,o3,2025-04-16,2025-04-16
66,m61,2.012738,o4-mini-2025-04-16_high,o4-mini,2025-04-16,2025-04-16
54,m50,2.012121,gemini-2.5-pro-exp-03-25,Gemini 2.5 Pro,2025-03-25,2025-03-25
81,m75,1.968745,o4-mini-2025-04-16_medium,o4-mini,2025-04-16,2025-04-16
...,...,...,...,...,...,...
95,m88,0.960343,Llama-2-7b,Llama 2-7B,2023-07-18,2023-07-18
93,m86,0.899685,gemma-2b,Gemma 2B,2024-02-21,2024-02-21
107,m99,0.859323,LLaMA-7B,LLaMA-7B,2023-02-24,2023-02-24
8,m106,0.820518,falcon-7b,Falcon-7B,2023-04-24,2023-04-24


In [10]:
df_db_anchor

,benchmark_id,estimated_difficulty,estimated_slope,benchmark_name,benchmark_release_date
0,b14,0.000000,1.000000,Winogrande,2019-07-24
1,b12,0.231773,1.463925,HellaSwag,2019-05-19
2,b10,0.319982,0.946426,TriviaQA,2017-05-09
3,b11,0.971537,0.899783,OpenBookQA,2018-09-08
4,b13,1.034700,2.126404,MMLU,2020-09-07
5,b15,1.173192,2.989367,BBH,2022-10-17
6,b22,1.285912,1.179336,GeoBench,2025-03-01
7,b3,1.555539,6.519499,MATH level 5,2021-03-05
8,b1,1.579506,3.193969,GPQA diamond,2023-11-20
9,b24,1.746870,3.240703,MCBench,2024-10-16


In [8]:
all_runs = {}           # will map benchmark_name -> dict of outputs
failed    = []          # keep track of anything that errors out

# --- loop --------------------------------------------------------------------
for _, row in df_db_anchor.iterrows():
    anchor_benchmark  = row['benchmark_name']       # e.g. "HellaSwag"
    anchor_difficulty = float(row['estimated_difficulty'])
    anchor_slope      = float(row['estimated_slope'])
    
    try:
        df, df_cm, df_db = fit_statistical_model(
            scores_df,
            anchor_benchmark  = anchor_benchmark,
            anchor_difficulty = anchor_difficulty,
            anchor_slope      = anchor_slope
        )
        all_runs[anchor_benchmark] = {
            "df1" : df,
            "df_cm1" : df_cm,
            "df_db" : df_db,
            # cache the anchor values for reference
            "anchor_difficulty" : anchor_difficulty,
            "anchor_slope"      : anchor_slope,
        }
    except Exception as e:
        failed.append((anchor_benchmark, str(e)))

# --- post-processing (optional) ----------------------------------------------
# 1) quick glance at what failed
if failed:
    print("Benchmarks that raised errors:", failed)

# 2) pull out the difficulty/slope re-estimates across all runs
summary = (
    pd.concat(
        {k: v["df_db"][["benchmark_name", "estimated_difficulty", "estimated_slope"]]
         for k, v in all_runs.items()},
        names=["anchor_benchmark"]
    )
    .reset_index(level=0)
)


`ftol` termination condition is satisfied.
Function evaluations 36, initial cost 2.3912e+01, final cost 1.6297e+00, first-order optimality 7.03e-05.
`ftol` termination condition is satisfied.
Function evaluations 36, initial cost 2.3912e+01, final cost 1.6297e+00, first-order optimality 6.70e-05.
`ftol` termination condition is satisfied.
Function evaluations 36, initial cost 2.3912e+01, final cost 1.6297e+00, first-order optimality 4.96e-05.
`ftol` termination condition is satisfied.
Function evaluations 48, initial cost 2.3912e+01, final cost 1.6297e+00, first-order optimality 2.14e-05.
`ftol` termination condition is satisfied.
Function evaluations 39, initial cost 2.3912e+01, final cost 1.6297e+00, first-order optimality 4.31e-05.
`ftol` termination condition is satisfied.
Function evaluations 39, initial cost 2.3912e+01, final cost 1.6297e+00, first-order optimality 7.41e-06.
`ftol` termination condition is satisfied.
Function evaluations 31, initial cost 2.3912e+01, final cost 1.

In [12]:
import pandas as pd
import numpy as np

# ---------------------------------------------------------------------------
# 1)  DIFFICULTY  ––  variation of each benchmark’s difficulty estimate
# ---------------------------------------------------------------------------
difficulty_rows = []

for anchor, run in all_runs.items():
    df_db = run["df_db"]                                  # difficulty table from that fit
    out = df_db[["benchmark_name", "estimated_difficulty"]].copy()
    out["anchor_benchmark"] = anchor                      # remember which fit this came from
    difficulty_rows.append(out)

difficulty_long = pd.concat(difficulty_rows, ignore_index=True)

# drop the trivial row where the benchmark was forced to be the anchor (always fixed):
difficulty_long = difficulty_long[
    difficulty_long["benchmark_name"] != difficulty_long["anchor_benchmark"]
]

difficulty_stats = (
    difficulty_long
      .groupby("benchmark_name")["estimated_difficulty"]
      .agg(mean   = "mean",
           std    = "std",
           min    = "min",
           max    = "max",
           cv     = lambda s: s.std(ddof=0) / s.mean()      # coefficient of variation
      )
      .sort_index()
)

# ---------------------------------------------------------------------------
# 2)  CAPABILITY  ––  variation of each model’s capability estimate
# ---------------------------------------------------------------------------
capability_rows = []

for anchor, run in all_runs.items():
    df_cm = run["df_cm1"]                                  # capability table from that fit
    out = df_cm[["model", "estimated_capability"]].copy()
    out["anchor_benchmark"] = anchor
    capability_rows.append(out)

capability_long = pd.concat(capability_rows, ignore_index=True)

capability_stats = (
    capability_long
      .groupby("model")["estimated_capability"]
      .agg(mean = "mean",
           std  = "std",
           min  = "min",
           max  = "max",
           cv   = lambda s: s.std(ddof=0) / s.mean()
      )
      .sort_index()
)

# ---------------------------------------------------------------------------
# 3)  quick look
# ---------------------------------------------------------------------------
print("=== variation in benchmark difficulties ===")
display(difficulty_stats)

print("\n=== variation in model capabilities ===")
display(capability_stats)


=== variation in benchmark difficulties ===


,mean,std,min,max,cv
benchmark_name,,,,,
ARC-AGI,1.991296,0.005605,1.965919,1.993459,0.002755
Aider polyglot,1.824987,0.033441,1.667988,1.832430,0.017938
BBH,1.142977,0.147998,0.448147,1.173311,0.126759
Balrog,2.330234,0.051432,2.318044,2.571690,0.021607
CadEval,1.837963,0.031225,1.691403,1.845086,0.016631
Cybench,2.408597,0.064566,2.395184,2.711721,0.026242
DeepResearch Bench,1.680332,0.170824,1.006038,1.872242,0.099520
Factorio learning environment,2.718053,0.116652,2.687400,3.265673,0.042014
FrontierMath-2025-02-28-Private,2.286417,0.044368,2.263628,2.494260,0.018997



=== variation in model capabilities ===


,mean,std,min,max,cv
model,,,,,
DeepSeek-R1,1.845659,0.029399,1.704547,1.852190,0.015607
DeepSeek-R1-0528,1.883756,0.023112,1.772831,1.889096,0.012021
DeepSeek-V2,1.623385,0.066045,1.306368,1.636945,0.039862
DeepSeek-V3,1.688087,0.055381,1.422259,1.699614,0.032144
DeepSeek-V3-0324,1.793668,0.037977,1.611379,1.801833,0.020745
...,...,...,...,...,...
qwen-plus-2025-01-25,1.620558,0.066502,1.301347,1.634263,0.040208
qwen2.5-32b-instruct,1.556272,0.077097,1.186209,1.572032,0.048538
qwen2.5-72b-instruct,1.602870,0.069417,1.269667,1.617139,0.042433
